<a href="https://colab.research.google.com/github/Zhi-704/ERL/blob/master/Genetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install jumanji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.5/583.5 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for jumanji: filename=jumanji-0.3.1-py3-none-any.whl size=750375 sha256=ac139eb013f600dbe3258b130755093b8e748ed9c7f4a3d00ce217689943a010
  Stored in directory: /root/.cache/pip/wheels/31/29/10/884df072d319d2522cfff8b9c3d61fe60ab3c31e2a054d3e5d
Successfully built jumanji


In [ ]:
import jumanji
import jax.numpy as jnp
import jax
import random
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
from time import sleep

In [ ]:
# Useful Code
'''

def random_action(self, state):
  array = np.array(state.action_mask)
  true_indices = self.get_valid_actions(state)
  if len(true_indices) == 0:
    return False
  random_index = np.random.choice(true_indices)
  # Convert the random index to two-dimensional coordinates
  rotation, col_index = np.unravel_index(random_index, array.shape)
  return [rotation,col_index]
  '''

In [ ]:
'''
GA()
   initialize population
   find fitness of population

   while (termination criteria is reached) do
      parent selection
      crossover with probability pc
      mutation with probability pm
      decode and fitness calculation
      survivor selection
      find best
   return best
'''

In [ ]:
# Helper functions

def convert_state(state):
  '''
  Convert state into observation variable that can be passed into neural network
  PARAM -
  state: current game state
  RETURNS - observation variable
  '''
  grid = state.grid_padded.flatten().tolist()
  tetromino = state.tetromino_index.flatten().tolist()
  obs_variable = np.asarray(grid+tetromino)
  # print("Current input dimensions:")
  # print(np.shape(obs_variable))
  # print(obs_variable)
  return obs_variable

def unravel(state, action_index):
  array = np.array(state.action_mask)
  rotation, col_index = np.unravel_index(action_index, array.shape)
  action = [rotation, col_index]
  return action

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class Population:

  def __init__(self, pop_size, p_mutation, p_crossover, p_selection, max_generations):
    self.pop_size = pop_size
    self.p_mutation = p_mutation
    self.p_crossover = p_crossover
    self.p_selection = p_selection
    self.max_generations = max_generations
    self.population = []
    self.elites = []

  def breed(self):
    '''
    Breeds initial population of solutions
    '''
    for i in range(self.pop_size):
      new_genome = Genome(170,64,40)
      self.population.append(new_genome)

  def parent_crossover(parent_a, parent_b):
    '''
    Creates another genome based off two parents where their weighting and biases mixed between the two in a one point crossover
    '''
    offspring = Genome(170,64,40)
    # Get weight and biases of parents hidden layer
    weights1, biases1 = parent_a.model.layers[1].get_weights()
    weights2, biases2 = parent_b.model.layers[1].get_weights()
    # Swap weights between parents to create the offspring
    midpoint_w = len(weights1) // 2
    midpoint_b = len(biases1) // 2
    # Combine weights and biases from parents up to the midpoint
    offspring_weights = np.concatenate((weights1[:midpoint_w], weights2[midpoint_w:]), axis=0)
    offspring_biases = np.concatenate((biases1[:midpoint_b], biases2[midpoint_b:]), axis=0)
    # Set weights of offspring
    offspring.model.layers[1].set_weights([offspring_weights, offspring_biases])
    return offspring

  def mutation(self, mutation_rate):
    '''
    Mutates a random agent where at each weight a dice is rolled whether or not the weighting changes
    '''
    # Choose random agent to mutate
    max_index = len(self.population)
    rand_num = random.randint(0, max_index -1)
    current_genome = self.population[rand_num]

    # Chooses random weight to mutate
    model = current_genome.model
    hidden_layer = model.layers[1]  # Assuming hidden layer is the second layer

    weights, biases = hidden_layer.get_weights()

    for i in range(len(weights)):
      for j in range(len(weights[i])):
        if np.random.rand() < mutation_rate:
          # Apply mutation to the weight
          weights[i][j] += np.random.normal(-0.1, 0.1)  # You can adjust the mutation strategy here

    current_genome.model.layers[1].set_weights([weights, biases])

    return

  def selection(self):
    return

  def total_fitness(self):
    return

  def run_gen(self):
    '''
    Run experience for all genomes in the population to calculate their fitness scores
    '''
    counter = 0

    for curr_genome in self.population:
      # Instantiate tetris environment using registry
      env = jumanji.make('Tetris-v0', time_limit = 500)
      key = jax.random.PRNGKey(1)
      state, timestep = jax.jit(env.reset)(key)

      # Run a whole episode for a genome
      while True:
        action = curr_genome.policy(state)
        if action is False:
          break
        next_state, next_timestep = jax.jit(env.step)(state, action)
        curr_genome.fitness += next_state.reward
        state = next_state

      counter += 1
      print("Genome {0} has been completed with fitness {1}".format(counter, curr_genome.fitness))


class Genome:
  def __init__(self, input_size, hidden_size, output_size):
    self.model = self.build_model(input_size, hidden_size, output_size)
    self.fitness = 0

  def build_model(self, input_size, hidden_size, output_size):
    # 128, 64, 40 input, hidden,output
    # 256, 128, 40 new

    # Vary the weights and biases for each genome
    kernel_init = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5)
    bias_init = initializer = tf.keras.initializers.RandomUniform(minval=-0.1, maxval=0.1)
    model = Sequential()
    # Adds fully connected layer with 128 units and uses rectified linear unit activation function. he_uniform initliazes weight of layer
    model.add(Dense(256, input_dim = input_size, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(hidden_size, activation='relu', use_bias=True, bias_initializer = bias_init, kernel_initializer= kernel_init))
    # 40 actions so 40 different outputs
    model.add(Dense(output_size, activation='linear', kernel_initializer='he_uniform'))
    # opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer = 'adam')
    return model

  def policy(self, state):
    # Matches state into 170(for 10x10), array
    state_array = convert_state(state)
    # Convert to into tensorflow tensor
    state_input = tf.convert_to_tensor(state_array[None, :], dtype=tf.float32)
    # Grabs Q values for all possible actions in current state
    action_q = np.array(self.model(state_input))
    # Allows only legal moves
    action_mask = np.array(state.action_mask).flatten()
    # Checks if terminal state is reached when there are no more legal moves
    terminal = np.where(action_mask)[0]
    if len(terminal) == 0:
      return False
    else:
      # Legal moves will have a value of 1 while illegal moves have a value of 0
      locater = action_mask.astype(int)
      # Ensures illegal moves remain illegal
      max_q = np.absolute(action_q).max()
      action_q[:,locater == 0] -= (max_q + 9999999999)
      # Grabs the index of the action and converts it to suitable type
      action_index = np.argmax(action_q)
      action = unravel(state, action_index)
      return action

In [ ]:
 pop = 5
 p_m = 0.1
 p_c = 0.2
 p_s = 0.3
 max_gen = 5

agent1 = Genome(170,64,40)
agent2 = Genome(170,64,40)

print("Starting population")
Big_poppa = Population(pop, p_m, p_c, p_s, max_gen)
Big_poppa.breed()
Big_poppa.run_gen()
print(len(Big_poppa.population))
print("Done")

# offspring = Population.parent_crossover(agent1, agent2)
# first_weights = offspring.model.layers[1].get_weights()[0]
# print("Printing first weights")
# print(first_weights)
# # Big_poppa.population.append(offspring)
# # Big_poppa.mutation(0.5)
# second_weights = offspring.model.layers[1].get_weights()[0]
# print("Printing second weights")
# print(second_weights)

Starting population
0
Done
Printing first weights
[[ 0.1034193  -0.10359573 -0.42488146 ... -0.13721013  0.4788549
   0.04425192]
 [ 0.17047215  0.12048793  0.13032424 ... -0.40349805 -0.04717028
  -0.08386469]
 [ 0.41368675 -0.25950205 -0.43391514 ...  0.20021594  0.08378482
  -0.39909458]
 ...
 [-0.41764235  0.46795082 -0.30122733 ...  0.27064657  0.04432058
  -0.21248484]
 [-0.46126747  0.4242971  -0.24358916 ... -0.01757932 -0.41838753
  -0.4259963 ]
 [-0.459324   -0.09271324 -0.0031389  ... -0.21763802  0.45194817
   0.07878995]]
Printing second weights
[[ 0.1034193  -0.10359573 -0.42488146 ... -0.13721013  0.4788549
   0.04425192]
 [ 0.17047215  0.12048793  0.13032424 ... -0.40349805 -0.04717028
  -0.08386469]
 [ 0.41368675 -0.25950205 -0.43391514 ...  0.20021594  0.08378482
  -0.39909458]
 ...
 [-0.41764235  0.46795082 -0.30122733 ...  0.27064657  0.04432058
  -0.21248484]
 [-0.46126747  0.4242971  -0.24358916 ... -0.01757932 -0.41838753
  -0.4259963 ]
 [-0.459324   -0.09271324 